In [1]:
from numpy import sin, cos, random as rn, array

In [2]:
'''
Defined Function
'''
def h(x1, x2):
    assert -1 <= x1 <= 2, 'Outside defined limit for x1!'
    assert -1 <= x2 <= 1, 'Outside defined limit for x2!'
    return cos(x1) * sin(x2) - x1 / (x2 ** 2 + 1)

In [3]:
'''
Used to decode chromosome
'''
def x(r_min, r_max, gens):
    assert r_min <= r_max, 'Minimum limit must be lesser than maximum limit!'
    g_sum = sum(gens)
    N = len(gens)
    return r_min + (r_max - r_min) * g_sum / N

In [4]:
'''
Generate chromosome based on random real number with size m,
for each population and n for the total of population
'''
def generate_population(m, n):
    assert m >= 0, 'Accepts positive integers!'
    assert m % 2 == 0, 'Accepts even integers!'
    return [[rn.random() for i in range(m)] for i in range(n)]

In [5]:
'''
Decode the chromosomes into two values, x1 and x2
'''
def decode(chromosome):
    N = len(chromosome)
    x1 = x(-1, 2, chromosome[0:N//2])
    x2 = x(-1, 1, chromosome[N//2:N])
    return x1, x2

In [6]:
'''
Returns minimum fitness value. If x1 and x2 is outside boundary,
then it will return 0 and determined formula if it is within.
'''
def fitness(x1, x2):
    try:
        return -h(x1, x2)
    except:
        return 0

In [7]:
'''
Calculate fitness of a chromosome
'''
def chromosome_fitness(chromosome):
    x1, x2 = decode(chromosome)
    return fitness(x1, x2)

In [8]:
'''
Evaluate fitness value of entire population
'''
def evaluate(populations):
    population_decode = [decode(p) for p in population]
    chromosome_fitness = [fitness(x1, x2) for x1, x2 in population_decode]
    result = sum(chromosome_fitness) 
    return result

In [9]:
'''
Sort genes based on its fitness value
'''
def sort_population(population):
    return sorted(population, key=lambda chromosome: chromosome_fitness(chromosome))

In [10]:
'''
Tournament selection for selecting best parents,
it will do k iteration to find values that is
suitable based on its fitness value
'''
def selection(population, k):
    population_copy = population.copy()
    best = []
    best_fitness = 0
    for i in range(k):
        random_index = rn.randint(len(population_copy))
        chromosome_individual = population_copy[random_index]
        fitness_individual = chromosome_fitness(chromosome_individual)
        if len(best) == 0 or fitness_individual > best_fitness:
            population_copy.pop(random_index)
            best.append(chromosome_individual)
            best_fitness = fitness_individual
    return best

In [11]:
'''
Crossover for population or parents
'''
def crossover(population, pc):
    N = len(population[0])
    population_cross = []
    while len(population_cross) < len(population):
        cross_probability = rn.randint(0, 101)
        i = len(population_cross)
        chromosome_new = population[i].copy()
        random_i = rn.randint(len(population))
        if cross_probability <= pc and random_i != i:            
            chromosome_random = population[random_i].copy()
            chromosome_new[0:N//2-1], chromosome_random[0:N//2+1] = chromosome_random[N//2+1:N], chromosome_new[N//2-1:N]
        population_cross.append(chromosome_new)
    return population_cross

In [12]:
'''
Mutation for population or offspring
'''
def mutation(population, pm):
    population_mutate = []
    N = len(population[0])
    for i in range(len(population)):
        probability_mutation = rn.randint(0, 101)
        chromosome = population[i].copy()
        if probability_mutation <= pm:
            random_i = rn.randint(N)
            chromosome[random_i] = rn.random()
        population_mutate.append(chromosome)
    return population_mutate

In [13]:
'''
Elitism is used to fetch k chromosomes with the best fitness value
'''
def elitism(population, k):
    population_sort = sort_population(population)
    return population_sort[-k:]

In [14]:
'''
Generational Replacement for Genetic Algorithm
'''
def generational_replacement_ga(max_count, selection_count, chromosome_count, gene_count, elite_count, pc, pm):
    population = generate_population(chromosome_count, gene_count)
    for i in range(max_count):
        new_population = elitism(population, elite_count)
        while len(new_population) < gene_count:
            parents = selection(population, selection_count)
            offspring = crossover(parents, pc)
            offspring = mutation(offspring, pm)
            new_population.extend(offspring)
        population = new_population
    best = max(population, key=lambda chromosome: chromosome_fitness(chromosome))
    x1, x2 = decode(best)
    return x1, x2, h(x1, x2)

generational_replacement_ga(400, 5, 6, 50, 2, 65, 1)

(1.9194284260772618, 0.08926172235651042, -1.9347084163735118)

In [15]:
'''
Steady state replaces the population with worst fitness value
with offsprings from parent selection
'''
def steady_state(population, offspring):
    N = len(offspring)
    population_worst = sort_population(population)[N:]
    for i in range(N):
        population.remove(population_worst[i])
        population.append(offspring[i])
    return population

In [16]:
'''
Fitness Based Steady State for Genetic Algorithm
'''
def steady_state_ga(max_count, selection_count, chromosome_count, gene_count, pc, pm):
    population = generate_population(chromosome_count, gene_count)
    for i in range(max_count):
        parents = selection(population, selection_count)
        offspring = crossover(parents, pc)
        offspring = mutation(parents, pm)
        population = steady_state(population, offspring)
    best = max(population, key=lambda chromosome: chromosome_fitness(chromosome))
    x1, x2 = decode(best)
    return x1, x2, h(x1, x2)

steady_state_ga(4000, 5, 6, 50, 65, 1)

(1.7903672821722818, 0.05615480168140263, -1.7969640540395928)